In [1]:
from read import read
from db import mk_db, count_words, words as get_words
from file import save, save_idfs
from preprocess import preprocess
from vectorize import tfidf_vectorizer, count_vectorizer, binary_vectorizer, vectorize, idf_vectorizer
from metrics import calc_metrics, cossine, jaccard, dice, euclide
from rocchio import rocchio
from classification import naive_bayes
from utils import merge
from ref import koefs, keywords, relevant_sents

In [2]:
import os
import ipywidgets as ws
import pandas as pd
from IPython.display import display

In [3]:
file_loader = ws.FileUpload(multiple=False)
drop = ws.Dropdown(options=[("Бин.", binary_vectorizer), ("TF", count_vectorizer), ("TF-IDF", tfidf_vectorizer)], description="Векторизатор:")
btn1 = ws.Button(description="Построить модель")
btn2 = ws.Button(description="Показать результаты")
btn3 = ws.Button(description="Поиск")
text = ws.Textarea()
out = ws.Output()

In [4]:
model, vectorizer, db = None, None, None

In [5]:
def on_btn_clicked(btn):
    global model
    global vectorizer
    global db
    with out:
        out.clear_output()
        ext = os.path.splitext(list(file_loader.value.keys())[0])[1]
        fn = ".tmp" + ext
        with open(fn, "wb") as f:
            f.write(list(file_loader.value.values())[0]['content'])
        model, vectorizer, db = vectorize(drop.value, preprocess(read(fn)))
        save(db, "data/.tmp.1db.csv")
        os.remove(fn)
        display("Индексирование завершено!!!")

def show_btn_clicked(btn):
    with out:
        out.clear_output()
        tmp = list(model)[:10]
        tmp = list(
            {
                item: [text.get(item, 0) for text in tmp]
                for item in list(tmp[0].keys())[:30]
            }.items()
        )
        tmp.sort(key=lambda x: x[1][0], reverse=True)
        
        display(pd.DataFrame(dict(tmp[:10])).transpose())

def metrics_btn_clicked(btn):
    with out:
        out.clear_output()
        words = {
            cossine: "Косинусная",
            jaccard: "Жаккара",
            dice: "Дайса"
        }
        res = pd.DataFrame({
            words[metric]: calc_metrics(metric, list(preprocess([text.value]))[0], model, vectorizer, get_words(db)) for metric in (cossine, jaccard, dice)
        })
        display(res)

btn1.on_click(on_btn_clicked)
btn2.on_click(show_btn_clicked)
btn3.on_click(metrics_btn_clicked)

In [6]:
display(file_loader, drop, btn1, btn2, text, btn3, out)

FileUpload(value={}, description='Upload')

Dropdown(description='Векторизатор:', options=(('Бин.', <function binary_vectorizer at 0x7ff93d5d3280>), ('TF'…

Button(description='Построить модель', style=ButtonStyle())

Button(description='Показать результаты', style=ButtonStyle())

Textarea(value='')

Button(description='Поиск', style=ButtonStyle())

Output()

In [7]:
import preprocess as p
import json

with open("data/stopwords.json") as f:
    stop_words = json.load(f)

p.stop_words = stop_words
p.preprocessors.append(p.rm_stop_words)

In [8]:
file_loader2 = ws.FileUpload(multiple=False)
btn6 = ws.Button(description="Предсказать")
out3 = ws.Output()

In [9]:
first_class = list(merge(read("data/lab3/train/C1_1.txt"), read("data/lab3/train/C1_2.txt"), read("data/lab3/train/C1_3.txt")))
second_class = list(merge(read("data/lab3/train/C2_1.txt"), read("data/lab3/train/C2_2.txt"), read("data/lab3/train/C2_3.txt")))
third_class = list(merge(read("data/lab3/train/C3_1.txt"), read("data/lab3/train/C3_2.txt"), read("data/lab3/train/C3_3.txt")))
classes = (
    range(len(first_class)),
    range(len(first_class), len(first_class) + len(second_class)),
    range(len(first_class) + len(second_class), len(first_class) + len(second_class) + len(third_class)),
)
_, _, nbc_db = vectorize(tfidf_vectorizer, p.preprocess(first_class + second_class + third_class))

In [10]:
def predict_btn(btn):
    with out3:
        out3.clear_output()
        ext = os.path.splitext(list(file_loader2.value.keys())[0])[1]
        fn = ".tmp" + ext
        with open(fn, "wb") as f:
            f.write(list(file_loader2.value.values())[0]['content'])
        a = list(naive_bayes(nbc_db, list(p.preprocess(read(fn)))[0], classes))
        print(a)
        print(max(range(len(a)), key=lambda i: a[i]) + 1)

btn6.on_click(predict_btn)

In [11]:
display(file_loader2, btn6, out3)

FileUpload(value={}, description='Upload')

Button(description='Предсказать', style=ButtonStyle())

Output()

In [12]:
text2 = ws.Textarea()
btn4 = ws.Button(description="Подобрать")
out2 = ws.Output()

In [13]:
sport_texts = list(read("data/sport_all.txt"))
tech_texts = list(read("data/tech_all.txt"))
class_names = ("Спорт", "Технологии")
classes = (range(len(sport_texts)), range(len(sport_texts), len(sport_texts) + len(tech_texts)))
rocc_model, rocc_vec, rocc_db = vectorize(tfidf_vectorizer, p.preprocess(sport_texts + tech_texts))
idfs = idf_vectorizer(rocc_db)
save(rocc_model, "data/.tmp.4model.csv")
save(rocc_db, "data/.tmp.4db.csv")
save_idfs(idfs, "data/.tmp.4idfs.json")

In [14]:
def rocchio_test(btn):
    with out2:
        out2.clear_output()
        res = pd.DataFrame(zip(class_names, tuple(
            rocchio(
                list(p.preprocess([text2.value]))[0],
                rocc_model,
                words(rocc_db),
                rocc_vec,
                classes,
                euclide,
            )
        )),columns=("Класс", "Расстояние"))
        display(res.sort_values(by="Расстояние"))

btn4.on_click(rocchio_test)

In [15]:
display(text2, btn4, out2)

Textarea(value='')

Button(description='Подобрать', style=ButtonStyle())

Output()

In [16]:
textl = ws.Textarea()
btnl = ws.Button(description="Process")
outl = ws.Output()

In [17]:
def on_clk(btn):
    with outl:
        outl.clear_output()
        ps = list(filter(lambda x: x != "", textl.value.split("\n")))
        db = list(mk_db(*count_words(p.preprocess(ps))))
        k = dict(filter(lambda el: el[1][1], sorted(koefs(db).items(), key=lambda el: el[1], reverse=True)))
        kw = keywords("\n".join(ps), db, k)
        s = relevant_sents("\n".join(ps), db, k)
        display(pd.DataFrame([(k, v[0], "ГОС" if v[1] == 1 else "ВОС" if v[1] == 2 else "Н") for k, v in k.items()]), pd.DataFrame(kw), pd.DataFrame(s.items()).sort_values(by=1, ascending=False)[:3].sort_index())
btnl.on_click(on_clk)


In [18]:
display(textl, btnl, outl)

Textarea(value='')

Button(description='Process', style=ButtonStyle())

Output()